In [248]:
#notebook of examples for the NTRU encryption scheme
#see pp. \S 7.10 and pp. 490-494 Hoffstein, Pipher, Silverman, An Introduction to Mathematical Cryptography

In [163]:
#define the parameters
N=5; p=3; q=32; d=1
assert N.is_prime()
assert p.is_prime()
assert gcd(N,q) == 1 
assert gcd(p,q) == 1
assert q > (6*d+1)*p

In [164]:
#create the polynomial ring \Z[x]/(x^N-1)
P.<x> = PolynomialRing(ZZ)
R = P.quotient(x^N-1); R

Univariate Quotient Polynomial Ring in xbar over Integer Ring with modulus x^5 - 1

In [165]:
#create the quotient polynomial ring \Z_p[x]/(x^N-1)
P_p.<x> = PolynomialRing(GF(p))
R_p = P_p.quotient(x^N - 1); R_p

Univariate Quotient Polynomial Ring in xbar over Finite Field of size 3 with modulus x^5 + 2

In [166]:
#create the quotient polynomial ring \Z_q[x]/(x^N-1)
P_q.<x> = PolynomialRing(GF(q))
R_q = P_q.quotient(x^N - 1); R_q

Univariate Quotient Polynomial Ring in xbar over Finite Field in z5 of size 2^5 with modulus x^5 + 1

In [180]:
#define the polynomial f(x). f(x) \in \mathcal{T}(d+1,d), i.e. is a ternary polynomial
#so has d+1 coeffs = 1, d coeffs = -1, all other coeffs = 0
f = R([1,1,-1,0,0])
print(f)
print(R_p(f))
print(R_q(f))

-xbar^2 + xbar + 1
2*xbar^2 + xbar + 1
xbar^2 + xbar + 1


In [181]:
#define the polynomial g(x). g(x) \in \mathcal{T}(d,d), i.e. is a ternary polynomial
#so has d coeffs = 1, d coeffs = -1, all other coeffs = 0
g = R([0,1,-1,0,0])
print(g)
print(R_p(g))
print(R_q(g))

-xbar^2 + xbar
2*xbar^2 + xbar
xbar^2 + xbar


In [183]:
#compute F_p(x) = f(x)^{-1} (mod p)
F_p = R_p(f)^(-1); print(F_p)
assert F_p * R_p(f) == 1

xbar^4 + 2*xbar^3 + 2*xbar + 2


In [184]:
#compute F_q(x) = f(x)^{-1} (mod q)
F_q = R_q(f)^(-1); print(F_q)
assert F_q * R_q(f) == 1

xbar^4 + xbar^2 + xbar


In [185]:
#define the public key using f(x) (mod p), g(x) (mod p)
def public_key(f,g,q):
    F_q = R_q(f)^(-1)
    return F_q * R_q(g)

In [189]:
#define the private key used to decrypt messages
def private_key(f):
    return (f,R_q(f)^(-1))

In [261]:
#define a plaintext message m(x) with coefficients satisfying -p/2 < m_i <= p/2
#i.e. m is the center lift of a polynomial in R_p
def plaintext(message,p,N):
    assert all([-p/2 < message[i] <= +p/2 for i in range(N)])
    return R(message)

In [257]:
#choose a random element of \mathcal{T}(d,d)
def rand_r(N,d):
    coeffs = (N-2*d)*[0] + d*[+1] + d*[-1]
    sigma = Permutations(N).random_element()
    return R([coeffs[sigma[i]-1] for i in range(N)])

In [259]:
#compute the ciphertext e(x)
def ciphertext(message,f,g,p,q,N,d):
    h = public_key(f,g,q)
    r = rand_r(N,d)
    m = plaintext(message,p,N)
    e = R_q(p*h*r + m)
    return e

In [326]:
message = [+1,-1,-1,0,0]

In [327]:
e = ciphertext(message,f,g,p,q,N,d)

In [264]:
#begin decryption process
def a(f,e):
    return R_q(f*e)

In [288]:
a(f,e)

xbar^2 + xbar + 1

In [323]:
center_lift(a(f,e))

xbar^2 + xbar + 1

In [322]:
#define the center lift of a from R_q to an element of R = \Z[x]/(x^N-1)
#map coefficients from F_q to Z/qZ (the additive group) and center lift them to \Z
def center_lift(a):
    return R([Zmod(coeff.order())(coeff).lift_centered() for coeff in list(a)])

In [324]:
#compute b(x) = F_p(x) * a(x) (mod p)
def b(f,a,p):
    F_p = R_p(f)^(-1)
    return R_p(F_p * center_lift(a))

In [325]:
b(f,a(f,e),p)

xbar^3 + xbar^2 + 2*xbar + 2